## Sonification of Pianorolls


In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from IPython import display as ipd

from libsoni.core import pianoroll
from libsoni.util.utils import mix_sonification_and_original, get_preset, visualize_piano_roll

Fs= 22050

### Simple Scenario

In [ ]:
c_maj_df = pd.read_csv(os.path.join('data_csv','demo_pianoroll','C_major_triad_pianoroll.csv'),delimiter=';')
c_maj_df

In [ ]:
visualize_piano_roll(c_maj_df);

In [ ]:
sonification_c_maj = pianoroll.sonify_pianoroll_clicks(c_maj_df)
print('Sonified C Major triad with clicks:')
ipd.display(ipd.Audio(sonification_c_maj, rate=Fs))

In [ ]:
preset = get_preset('bassoon')
sonification_c_maj = pianoroll.sonify_pianoroll_additive_synthesis(c_maj_df,
                                                                  partials=preset['partials'],
                                                                  partials_amplitudes=preset['amplitudes'])



print('Sonified C Major triad with additive synthesis:')
ipd.display(ipd.Audio(sonification_c_maj, rate=Fs))

In [ ]:
piano_c_60_sample, _ = librosa.load(os.path.join('data_audio','samples','01Pia1F060f_np___0_short.wav'), sr=Fs)

sonification_c_maj_sample = pianoroll.sonify_pianoroll_sample(c_maj_df,
                                                              sample=piano_c_60_sample,
                                                              reference_pitch=60)
print('Sonified C Major triad with warped samples:')
ipd.display(ipd.Audio(sonification_c_maj_sample, rate=Fs))

## Bach Fugue in C Major, BWV 846

In [ ]:
bach_df = pd.read_csv(os.path.join('data_csv','demo_pianoroll','FMP_C1_F12_Bach_BWV846_Sibelius-Tracks.csv'),delimiter=';')
visualize_piano_roll(bach_df, figsize=(10, 7), colors='gist_rainbow');

In [ ]:
sonification_bach = pianoroll.sonify_pianoroll_clicks(bach_df)
ipd.display(ipd.Audio(sonification_bach, rate=Fs))

In [ ]:
sonification_c_maj_sample = pianoroll.sonify_pianoroll_sample(bach_df,
                                                              sample=piano_c_60_sample,
                                                              reference_pitch=60)
print('Beethoven Sonification with piano sample:')
ipd.display(ipd.Audio(sonification_c_maj_sample, rate=Fs))